In [5]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [6]:
# Data Directory
data_dir = 'dataset/'

# Load the data
train_1 = pd.read_csv(data_dir + 'train_1.csv')
train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')
test_1 = pd.read_csv(data_dir + 'test_1.csv')
test_2_1 = pd.read_csv(data_dir + 'test_2_1.csv')
test_2_2 = pd.read_csv(data_dir + 'test_2_2.csv')

/tmp/ipykernel_16663/3751997310.py:5: DtypeWarning: Columns (126,128,143) have mixed types. Specify dtype option on import or set low_memory=False.
  train_1 = pd.read_csv(data_dir + 'train_1.csv')
/tmp/ipykernel_16663/3751997310.py:6: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
/tmp/ipykernel_16663/3751997310.py:7: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')


In [7]:
# Remove duplicates based on 'id' in the additional information DataFrames
train_2_1 = train_2_1.drop_duplicates(subset='id')
train_2_2 = train_2_2.drop_duplicates(subset='id')
test_2_1 = test_2_1.drop_duplicates(subset='id')
test_2_2 = test_2_2.drop_duplicates(subset='id')

# Merge the DataFrames
train = pd.merge(train_1, train_2_1, on='id', how='left')
train = pd.merge(train, train_2_2, on='id', how='left')

test = pd.merge(test_1, test_2_1, on='id', how='left')
test = pd.merge(test, test_2_2, on='id', how='left')

In [8]:
# Release memory by deleting unnecessary variables and forcing garbage collection
del  test_2_1, test_2_2, train_1, train_2_1, train_2_2
gc.collect()

864

In [9]:
from scipy import stats

# Identify numerical and categorical columns
numerical_cols_train = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_train = train.select_dtypes(include=['object']).columns

numerical_cols_test = test.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_test = test.select_dtypes(include=['object']).columns

# Handle missing values for numerical columns train
for col in numerical_cols_train:
    train[col] = train[col].fillna(train[col].mean())
# Handle missing values for categorical columns train
for col in categorical_cols_train:
    train[col] = train[col].fillna('missing')

# Handle missing values for numerical columns test
for col in numerical_cols_test:
    test[col] = test[col].fillna(test[col].mean())
# Handle missing values for categorical columns test
for col in categorical_cols_test:
    test[col] = test[col].fillna('missing')

# # Handle outliers for numerical columns using Z-score
# for col in numerical_cols_train:
#     train = train[(np.abs(stats.zscore(train[col])) < 3)]
# for col in numerical_cols_test:
#     test = test[(np.abs(stats.zscore(test[col])) < 3)]

In [10]:
# Select relevant features
features = train.columns.drop(categorical_cols_train).drop('label')
X_train = train[features]
y_train = train['label']
X_test = test[features]


In [11]:
# Release memory by deleting unnecessary variables and forcing garbage collection
del train, test
gc.collect()

0

In [12]:
# # Remove 'loan_id' and 'id' from the list of categorical columns if they exist
# categorical_cols_train = [col for col in categorical_cols_train if col not in ['loan_id', 'id']]
# categorical_cols_test = [col for col in categorical_cols_test if col not in ['loan_id', 'id']]

# # Remove 'loan_id' and 'id' from the features if they exist
# X_train = X_train.drop(columns=['loan_id', 'id'], errors='ignore')
# X_test = X_test.drop(columns=['loan_id', 'id'], errors='ignore')

# # Convert categorical columns to numerical using one-hot encoding
# X_train = pd.get_dummies(X_train, columns=categorical_cols_train)
# X_test = pd.get_dummies(X_test, columns=categorical_cols_test)

# # Ensure the same columns in train and test after one-hot encoding
# X_train, X_test = X_train.align(X_test, join='inner', axis=1, fill_value=0)


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the XGBoost model
model = xgb.XGBClassifier(
    n_estimators=10000,        # Increase the number of boosting rounds
    learning_rate=0.01,       # Reduce the learning rate
    max_depth=20,             # Increase the maximum depth of trees
    subsample=0.8,            # Use subsampling
    colsample_bytree=0.8,     # Use column subsampling
    reg_alpha=0.1,            # Increase L1 regularization
    reg_lambda=1.0,           # Increase L2 regularization
    eval_metric='auc',        # Evaluation metric
    random_state=42           # Seed for reproducibility
)
model.fit(X_train, y_train)


In [15]:
# You can then make predictions on the test set
predictions = model.predict_proba(X_test)[:, 1]  # Get probabilities for class 1

# Ensure the lengths match
# predictions = predictions[:len(test_1)]

# Save predictions to a submission file
submission = pd.DataFrame({
    'loan_id': test_1['loan_id'],
    'prob': predictions
})
submission.to_csv('submission.csv', index=False)